## Data Preprocessing
In this section, data needs to be cleaned. Duplicates need to be removed, we need to ensure that the correct datatypes are being used. The data needs to be manipulated and formatted in a specific way so as to gain further insights.
The following needs to be done;


### Duplicate Analysis:
o Identify and remove duplicate rows, if there are any.

o Justify your approach.

### Feature Engineering:
o Convert Runtime (e.g., "142 min") to numerical (e.g., 142).

o Extract Decade from Released_Year (e.g., "1990s").

o Create a Lead_Actors column combining Star1, Star2, Star3, Star4.

In [13]:
# Necessary functions for executing the next cell
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

    # Check for missing values
class Data_Collection:
    def check_missing(df):
        """
        Function checks for missing values, counts the total number of missing values,
        compares them to the total number of values and prints the results.
        """
        df_check = df.isna()
        count1 = 0 # To count the number of missing and non missing values
        count2 = 0
        for col in df_check.columns:
            for row in df_check[col]:
                if row == True: # Returns a missing value
                    count1 += 1
                    # print(row, col) # Shows where the None values are. NOTE They are in the 'parental_education_level'
                else:
                    count2 += 1# Returns a non missing value
        print('Number of missing values', count1)
        print('Total number of values in the table', count1 + count2)
        missing_percentage = 100.0 * count1 / (count1 + count2)
        print(f'Proportion of missing data to the total: {round(missing_percentage, 2)}% of data is missing.') # percentage is well below 1%, Unlikely to have a major impact on overall findings
        # print(df.isna()) # Returns missing values as true

    # (Function) Remove rows with critical missing data, critical data column specified
    def remove_rows(df, column): 
        for i, row in df.iterrows():
            if pd.isna(row[column]): # How to equate to None/NaN in pandas. Pandas has no None, it's NaN in pandas
                # print(1)
                df.drop(index= i, inplace= True) #NOTE Inplace = True changes the DF
        return df

    # (Function) Remove duplicates
    def dropDuplicates(df):
        df_noDuplicates = df.drop_duplicates() # Removes any row that is a duplicate of any other, leaving only the first occurance
        return df_noDuplicates

class Data_Preperation:
    # (Funtion) Creating a new 'Relesed_Decade' column from the 'Released_Year' column
    def newColumn_ReleasedDecade(df):
        # Convert 'Released_Year' from string to int
        df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors= 'coerce') # 'coerce' to make errors encountered NaN
        df['Released_Year'] = df['Released_Year'].fillna(0)
        df['Released_Year'] = df['Released_Year'].astype(int)
        # Extract Decade from Released_Year
        list1 = []
        for row in df['Released_Year']:
            if row == 0:
                list1.append('0')
            elif row >= 1920 and row < 1930:
                list1.append('1920s')
            elif row >= 1930 and row < 1940:
                list1.append('1930s')
            elif row >= 1940 and row < 1950:
                list1.append('1940s')
            elif row >= 1950 and row < 1960:
                list1.append('1950s')
            elif row >= 1960 and row < 1970:
                list1.append('1960s')
            elif row >= 1970 and row < 1980:
                list1.append('1970s')
            elif row >= 1980 and row < 1990:
                list1.append('1980s')
            elif row >= 1990 and row < 2000:
                list1.append('1990s')
            elif row >= 2000 and row < 2010:
                list1.append('2000s')
            elif row >= 2010 and row < 2020:
                list1.append('2010s')
            elif row >= 2020 and row < 2030:
                list1.append('2020s')
        df_col = pd.DataFrame({'Released_Decade': list1})
        df_res = pd.concat([df, df_col], axis= 1) # Concat the decade column to the main DF
        return df_res


In [26]:
# Prerequisite code
df = pd.read_csv('Assignment_2/imdb_top_1000.csv') # Doesn't want to read imdb file in the .cache folder. Had to move it to the local folder

# Check for mising values and document their percantage
    # Using functions written into functions_Assignment_2.py file
missing = Data_Collection.check_missing(df) 

# Remove rows with missing critical data, from the following columns
df_new = Data_Collection.remove_rows(df, 'Meta_score')
df_new = Data_Collection.remove_rows(df_new, 'Gross')
print('\nAfter removal of null values')
missing_df_new = Data_Collection.check_missing(df_new)
#-----------------------------------------------------------------------------------------------------------------------------------------

df_noDuplicates = Data_Collection.dropDuplicates(df_new)

# Convert runtime to numerical
newRuntime = []
cnt = 0
for row in df_noDuplicates['Runtime']:
    newRuntime.append(float(row.split(' ')[0])) # New list of numerical values only
    
# Reset indices so there are no skipped values
df_reset = df_noDuplicates
df_reset.reset_index(inplace= True) #NOTE Index values are now reset for all affilliated DFs being used from now on

df_noDuplicates['Runtime'] = pd.DataFrame({'Runtime': newRuntime}) # Make newRuntime list a DF column by overriding the original
print('\nCheck Runtime column values dtype:')
print(df_noDuplicates['Runtime'].info())

# Extract Decade from Released_Year
df_res = Data_Preperation.newColumn_ReleasedDecade(df_noDuplicates)
print('\nFirst 5 entries in Released_Year and Released_Decade columns')
print(df_res[['Released_Year', 'Released_Decade']].head()) # It checks out

# Create a Lead_Actors column combining Star1, Star2, Star3, Star4.                          #NOTE IMPORTANT FUNCTION
df_res['Lead_Actors'] = df_res[['Star1', 'Star2', 'Star3', 'Star4']].agg(', '.join, axis= 1) # New column, all 4 cell values in one, using aggregate funtion where all values are joined along the columns axis
print('\nFirst 5 entries in new Lead_Actors column')
print(df_res['Lead_Actors'].head())


Number of missing values 427
Total number of values in the table 16000
Proportion of missing data to the total: 2.67% of data is missing.

After removal of null values
Number of missing values 36
Total number of values in the table 12000
Proportion of missing data to the total: 0.3% of data is missing.

Check Runtime column values dtype:
<class 'pandas.core.series.Series'>
RangeIndex: 750 entries, 0 to 749
Series name: Runtime
Non-Null Count  Dtype  
--------------  -----  
750 non-null    float64
dtypes: float64(1)
memory usage: 6.0 KB
None

First 5 entries in Released_Year and Released_Decade columns
   Released_Year Released_Decade
0           1994           1990s
1           1972           1970s
2           2008           2000s
3           1974           1970s
4           1957           1950s

First 5 entries in new Lead_Actors column
0    Tim Robbins, Morgan Freeman, Bob Gunton, Willi...
1    Marlon Brando, Al Pacino, James Caan, Diane Ke...
2    Christian Bale, Heath Ledger, Aaro